In [3]:
import numpy as np
import utils
from utils import loadmat
from extractDigitFeatures import extractDigitFeatures
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Conv2D, Conv3D, Flatten, MaxPool2D, AveragePooling2D, BatchNormalization
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import CSVLogger
from tensorflow.python.keras import metrics
from keras import regularizers
from keras.constraints import unit_norm
from sklearn.model_selection import StratifiedKFold
from keras.callbacks import EarlyStopping

Using TensorFlow backend.


In [4]:
def model_extractor(activation_func='relu', weight_decay=1e-7):
    # Creating an AlexNet Classifier
    model = Sequential()

    #Instantiating Layer 1
    model.add(Conv2D(48, kernel_size=(3, 3), strides=(1, 1), activation=activation_func, padding='valid', 
                    kernel_constraint=unit_norm(), kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(MaxPool2D(pool_size=(2, 2), strides=(2, 2)))

    # #Instantiating Layer 2
    model.add(Conv2D(96, kernel_size=(3, 3), strides=(1, 1), activation=activation_func, padding='same', 
                    kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(MaxPool2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(BatchNormalization())
    # #Instantiating Layer 3
    model.add(Conv2D(192, kernel_size=(3, 3), strides=(1, 1), activation=activation_func, padding='same',
                    kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(BatchNormalization())

    # #Instantiating Layer 4
    model.add(Conv2D(192, kernel_size=(3, 3), strides=(1, 1), activation=activation_func, padding='same', 
                    kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(MaxPool2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(BatchNormalization())

    # #Instantiating Layer 5
    model.add(Conv2D(256, kernel_size=(3, 3), strides=(1, 1), activation=activation_func, padding='same', 
                    kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(MaxPool2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(BatchNormalization())
    model.add(Flatten())

    #Instantiating Layer 6
    model.add(Dense(512, activation=activation_func)) 

    # #Instantiating Layer 8
    model.add(Dense(256, activation=activation_func))

    #Output Layer
    model.add(Dense(10, activation='softmax'))
    return model

In [5]:
def reformat(x):
    new_x = np.zeros((x.shape[2], x.shape[0], x.shape[1]))
    for i in range(x.shape[2]):
        new_x[i, :] = x[:, :, i]
    return new_x[:, :, :, np.newaxis]

def return_datasets(filename):
    data = utils.loadmat('../data/{}'.format(filename))
    trainSet, testSet, valSet = 1, 2, 3
    
    x_train = reformat(data['x'][:, :, data['set']==trainSet ])
    y_train = (data['y'][data['set']==trainSet])
    x_val = reformat(data['x'][:, :, data['set']==valSet])
    y_val = (data['y'][data['set']==valSet])
    x_test = reformat(data['x'][:, :, data['set']==testSet])
    y_test = (data['y'][data['set']==testSet])
    
    X = np.vstack([x_train, x_val])
    Y = np.vstack([y_train[:, np.newaxis], y_val[:, np.newaxis]])
    
    return (X, Y, x_val, y_val, x_test, y_test)

In [7]:
MAX_EPOCHS = 15
MAX_BATCH_SIZE = 200
MAX_TOLERANCE = 5

X, Y, x_val, y_val, x_test, y_test = return_datasets('digits-jitter.mat')

model = model_extractor('selu')
model.compile(loss="categorical_crossentropy", optimizer='Adam', validation_data=(x_val, y_val), metrics=["accuracy"])
model.fit(X, Y, epochs=MAX_EPOCHS, batch_size=MAX_BATCH_SIZE, verbose=1)
accuracy = model.evaluate(x_test, y_test, verbose=0)[1]

Train on 1500 samples
Epoch 1/15
1500/1500 [==============================] - 10s 6ms/sample - loss: 143.2442 - accuracy: 0.1800
Epoch 2/15
1500/1500 [==============================] - 4s 2ms/sample - loss: 271.0713 - accuracy: 0.2000
Epoch 3/15
1500/1500 [==============================] - 3s 2ms/sample - loss: 666.9118 - accuracy: 0.4660
Epoch 4/15
1500/1500 [==============================] - 3s 2ms/sample - loss: 1618.8665 - accuracy: 0.9360
Epoch 5/15
1500/1500 [==============================] - 4s 3ms/sample - loss: 3298.9869 - accuracy: 0.9767
Epoch 6/15
1500/1500 [==============================] - 5s 3ms/sample - loss: 6016.4472 - accuracy: 0.9907
Epoch 7/15
1500/1500 [==============================] - 5s 3ms/sample - loss: 9954.7412 - accuracy: 0.9947
Epoch 8/15
1500/1500 [==============================] - 5s 3ms/sample - loss: 15535.6141 - accuracy: 0.9940
Epoch 9/15
1500/1500 [==============================] - 5s 3ms/sample - loss: 22740.7344 - accuracy: 0.9933
Epoch 10/15
150

In [8]:
model.predict_classes(x_test)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [9]:
y_test

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6,